In [11]:
# Cell 2 — Dataset Download & Folder Setup

import os
import urllib.request
from pathlib import Path
import random

# ------------- VERIFIED WORKING DATA URLs -------------
# These URLs are stable, open-license, and accessible
classes = {
    "aspirin": [
        "https://images.pexels.com/photos/3683056/pexels-photo-3683056.jpeg",
        "https://images.pexels.com/photos/3683088/pexels-photo-3683088.jpeg",
        "https://images.pexels.com/photos/4210610/pexels-photo-4210610.jpeg"
    ],
    "warfarin": [
        "https://images.pexels.com/photos/159211/headache-pain-pills-medication-159211.jpeg",
        "https://images.pexels.com/photos/360622/pexels-photo-360622.jpeg",
        "https://images.pexels.com/photos/356040/pexels-photo-356040.jpeg"
    ],
    "unknown": [
        "https://images.pexels.com/photos/208518/pexels-photo-208518.jpeg",
        "https://images.pexels.com/photos/593451/pexels-photo-593451.jpeg",
        "https://images.pexels.com/photos/360622/pexels-photo-360622.jpeg"
    ]
}

# Number of duplicates per image (makes dataset larger)
COPIES = 10   # Creates about 30 images per class

# ------------- CREATE FOLDER STRUCTURE -------------

root = Path("data")
(train := root / "train").mkdir(parents=True, exist_ok=True)
(val := root / "val").mkdir(parents=True, exist_ok=True)

# Create folders for each class
for cls in classes:
    (train / cls).mkdir(parents=True, exist_ok=True)
    (val / cls).mkdir(parents=True, exist_ok=True)

# ------------- DOWNLOAD FUNCTION -------------

def download_image(url, save_path):
    """Download image with browser-like headers to avoid 403 errors"""
    try:
        req = urllib.request.Request(
            url,
            headers={"User-Agent": "Mozilla/5.0"}  # Spoofs a browser request
        )
        with urllib.request.urlopen(req) as resp:
            with open(save_path, "wb") as f:
                f.write(resp.read())
        print(f"Downloaded: {save_path}")
    except Exception as e:
        print(f"Failed: {url} -> {e}")

# ------------- DOWNLOAD ALL IMAGES -------------

for cls, urls in classes.items():
    for i, url in enumerate(urls):
        for c in range(COPIES):
            fname = f"{cls}_{i}_{c}.jpg"
            save_dir = train / cls if random.random() > 0.2 else val / cls   # 80/20 split
            save_path = save_dir / fname
            download_image(url, save_path)

print("\n🎉 Dataset downloaded successfully! You can now train the model.")


Downloaded: data/train/aspirin/aspirin_0_0.jpg
Downloaded: data/train/aspirin/aspirin_0_1.jpg
Downloaded: data/train/aspirin/aspirin_0_2.jpg
Downloaded: data/train/aspirin/aspirin_0_3.jpg
Downloaded: data/val/aspirin/aspirin_0_4.jpg
Downloaded: data/train/aspirin/aspirin_0_5.jpg
Downloaded: data/train/aspirin/aspirin_0_6.jpg
Downloaded: data/train/aspirin/aspirin_0_7.jpg
Downloaded: data/train/aspirin/aspirin_0_8.jpg
Downloaded: data/train/aspirin/aspirin_0_9.jpg
Downloaded: data/train/aspirin/aspirin_1_0.jpg
Downloaded: data/train/aspirin/aspirin_1_1.jpg
Downloaded: data/train/aspirin/aspirin_1_2.jpg
Downloaded: data/train/aspirin/aspirin_1_3.jpg
Downloaded: data/train/aspirin/aspirin_1_4.jpg
Downloaded: data/val/aspirin/aspirin_1_5.jpg
Downloaded: data/train/aspirin/aspirin_1_6.jpg
Downloaded: data/train/aspirin/aspirin_1_7.jpg
Downloaded: data/val/aspirin/aspirin_1_8.jpg
Downloaded: data/val/aspirin/aspirin_1_9.jpg
Downloaded: data/val/aspirin/aspirin_2_0.jpg
Downloaded: data/train/

In [16]:
# Cell 1: imports & config
import os
from pathlib import Path
import time
import threading
from dotenv import load_dotenv

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

import cv2
import schedule
import smtplib
from email.message import EmailMessage

# load environment variables
load_dotenv()
EMAIL_USER = os.getenv("EMAIL_USER")
EMAIL_PASS = os.getenv("EMAIL_PASS")
CAREGIVER_EMAIL = os.getenv("CAREGIVER_EMAIL")


In [19]:
# ============================================
# Cell 3 — Data Generators (Improved)
# ============================================

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

DATA_DIR = Path("data")  # expect data/train and data/val inside
IMG_SIZE = (224, 224)
BATCH = 16

train_dir = str(DATA_DIR / "train")
val_dir = str(DATA_DIR / "val")

# Safety check
assert (DATA_DIR / "train").exists(), "❌ Train folder not found. Run Cell 2 first!"
assert (DATA_DIR / "val").exists(), "❌ Val folder not found. Run Cell 2 first!"

# Training generator with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation generator (no augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    shuffle=True,        # IMPORTANT
    seed=42
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical',
    shuffle=False
)

# Display class mapping
class_indices = train_gen.class_indices
print("📌 Classes detected:", class_indices)


Found 90 images belonging to 3 classes.
Found 48 images belonging to 3 classes.
📌 Classes detected: {'aspirin': 0, 'unknown': 1, 'warfarin': 2}


In [20]:
# ============================================
# Cell 4 — MobileNetV2 Model (Improved)
# ============================================

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Load pretrained MobileNetV2
base = MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

# Freeze base layers (initial training)
base.trainable = False

# Build model
model = models.Sequential([
    layers.Input(shape=IMG_SIZE + (3,)),
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(len(class_indices), activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()
print("\n✔ Model is ready for training!")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)


✔ Model is ready for training!


In [22]:
# Cell 5: train
import os
from tensorflow.keras.models import load_model

EPOCHS = 10

# Train model
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen
)

# --- Auto-detect save path ---
current_dir = os.getcwd()              # Where the notebook is running
model_path = os.path.join(current_dir, "pill_classifier_mobilenetv2_ft.h5")

# --- Save the model ---
model.save(model_path)
print(f"Model saved at: {model_path}")

# --- Load the model safely ---
if os.path.exists(model_path):
    model = load_model(model_path)
    print("Model loaded successfully!")
else:
    print("ERROR: Model file not found:", model_path)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 55s 10s/step - accuracy: 0.5556 - loss: 0.9427 - val_accuracy: 0.7708 - val_loss: 0.7422
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 39s 7s/step - accuracy: 0.7333 - loss: 0.7226 - val_accuracy: 0.9167 - val_loss: 0.5704
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 36s 6s/step - accuracy: 0.7778 - loss: 0.5794 - val_accuracy: 0.9167 - val_loss: 0.4334
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 41s 7s/step - accuracy: 0.7778 - loss: 0.5380 - val_accuracy: 0.9167 - val_loss: 0.3506
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 45s 8s/step - accuracy: 0.8889 - loss: 0.3950 - val_accuracy: 0.9167 - val_loss: 0.2958
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 37s 6s/step - accuracy: 0.9000 - loss: 0.3938 - val_accuracy: 0.9167 - val_loss: 0.2622
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 46s 8s/step - accuracy: 0.8556 - loss: 0.3917 - val_accuracy: 0.9167 - val_loss: 0.2344
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 38s 7s/step - accuracy: 0.8667 - loss: 0.3403 - val_accuracy: 0.9167 - val_loss: 0.2174
Epoch 9

Model saved at: /home/buruk-bmt/Desktop/Machine Learning/Projects/AFIB-Detection/pill_classifier_mobilenetv2_ft.h5


Model loaded successfully!


In [ ]:
# Fine-tuning setup

# Unfreeze base model for fine-tuning
base.trainable = True

# Freeze first 100 layers (stabilizes training)
for layer in base.layers[:100]:
    layer.trainable = False

# Compile with smaller learning rate
model.compile(
    optimizer=Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.fit(train_gen, epochs=5, validation_data=val_gen)

model.save("pill_classifier_mobilenetv2_ft.h5")

model.summary()
print("Fine-tuning enabled. Ready to train further.")


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

model = load_model("pill_classifier_mobilenetv2_ft.h5")  # or ft model

inv_class_map = {v:k for k,v in class_indices.items()}

def predict_image(img_bgr):
    # img_bgr: numpy array BGR (OpenCV)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, IMG_SIZE)
    x = img_resized.astype("float32") / 255.0
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)
    idx = np.argmax(preds[0])
    conf = preds[0][idx]
    label = inv_class_map[idx]
    return label, float(conf)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'pill_classifier_mobilenetv2.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# Cell 7: webcam scanner (press 'q' to quit)
def run_scanner(threshold=0.7, display=True):
    cap = cv2.VideoCapture(0)  # 0 = default webcam
    if not cap.isOpened():
        raise RuntimeError("Could not open webcam")
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            label, conf = predict_image(frame)
            text = f"{label} ({conf:.2f})"
            if display:
                cv2.putText(frame, text, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
                cv2.imshow("Pill Scanner", frame)
            # decide detection event
            if conf >= threshold:
                print("Detected:", text)
                # You can return or call handler here
                # For demo: break when detection occurs
                # break
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

# To run:
# run_scanner()


In [ ]:
# Cell 8: reminders + local notification (console)
import schedule
import threading
import time
import smtplib
from email.message import EmailMessage

reminders = [
    {"time": "14:30", "pill": "warfarin"},
    {"time": "20:00", "pill": "aspirin"}
]

def reminder_job(pill_name):
    print(f"[Reminder] Time to take: {pill_name} — open scanner to confirm.")
    # Optionally pop a GUI alert (not shown here), or beep
    # Start scanner and wait for confirmation for N seconds
    confirmed = scanner_confirm(pill_name, timeout=30)  # returns True/False
    if not confirmed:
        send_caregiver_email(pill_name)

def scanner_confirm(expected_pill, timeout=30):
    """Run scanner for timeout seconds, return True if expected_pill detected."""
    cap = cv2.VideoCapture(0)
    start = time.time()
    try:
        while time.time() - start < timeout:
            ret, frame = cap.read()
            if not ret: break
            label, conf = predict_image(frame)
            if label == expected_pill and conf > 0.75:
                print("Patient confirmed took pill:", label)
                return True
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()
    print("No confirmation within timeout.")
    return False

# schedule these
for r in reminders:
    schedule.every().day.at(r["time"]).do(reminder_job, pill_name=r["pill"])

# run scheduler in background thread
def run_schedule_loop():
    while True:
        schedule.run_pending()
        time.sleep(1)

thread = threading.Thread(target=run_schedule_loop, daemon=True)
thread.start()
print("Reminder scheduler running in background.")


In [ ]:
# Cell 9: email notifier
def send_caregiver_email(pill_name):
    if not EMAIL_USER or not EMAIL_PASS or not CAREGIVER_EMAIL:
        print("Email credentials not configured.")
        return
    msg = EmailMessage()
    msg['Subject'] = f"CardiaGuard: Missed medication - {pill_name}"
    msg['From'] = EMAIL_USER
    msg['To'] = CAREGIVER_EMAIL
    body = f"Patient missed scheduled dose of {pill_name}. No confirmation from scanner."
    msg.set_content(body)
    # optional: attach last detected image or timestamp
    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(EMAIL_USER, EMAIL_PASS)
        server.send_message(msg)
        server.quit()
        print("Caregiver notified by email.")
    except Exception as e:
        print("Failed to send email:", e)
